# logging

**ログレベルの説明**
|レベル|数値|説明|
|--|--|--|
|CRITICAL|50|致命的な状態でサービス提供ができないようなときのログに設定する。|
|ERROR|40|不具合が起きているときのログに設定する。|
|WARNING|30|エラーではが、潜在的に問題になる可能性があるときのログに設定する。|
|INFO|20|通常処理の中であとから確認する可能性がある情報などのログに設定する。ジョブの開始と終了の時刻など。|
|DEBUG|10|開発中に確認する必要のある情報を出力するときのログに設定する。|
  
**フォーマット文字列**
|表記|意味|
|--|--|
|%(asctime)s|日時|
|%(levelname)s|ログレベル|
|%(filename)s|ファイル名|
|%(module)s|モジュール名|
|%(lineno)d|行番号|
|%(funcName)s|関数、メソッド名|
|%(message)s|メッセージ|
|%(name)s|ロガー名|

## 一般的な従来の1件1行のログを出力する

In [ ]:
from logging import (
    getLogger,
    StreamHandler,
    FileHandler,
    Formatter,
    DEBUG,
    INFO,
    WARNING,
    ERROR,
    CRITICAL
)

logger = getLogger(__name__)
# ロガーのログレベルを設定(ハンドラーよりも低い数値(DEBUGに近い)のログレベルにする必要がある)
logger.setLevel(DEBUG)

# フォーマットを設定
formatter = Formatter(
    "%(asctime)s - %(levelname)s : %(name)s : %(funcName)s : %(message)s (%(filename)s:%(lineno)d)"
)

# 標準出力用ハンドラーのログレベルを設定
s_handler = StreamHandler()
s_handler.setLevel(WARNING)
s_handler.setFormatter(formatter)
logger.addHandler(s_handler)

# ファイル出力用ハンドラーのログレベルを設定
f_handler = FileHandler(filename="./log/app.log", encoding="utf-8")
f_handler.setLevel(INFO)
f_handler.setFormatter(formatter)
logger.addHandler(f_handler)

# ログを出力
logger.critical("クリティカルのメッセージ")
logger.error("エラーのメッセージ")
logger.warning("ワーニングのメッセージ")
logger.info("インフォのメッセージ")
logger.debug("デバッグのメッセージ")

## JSON形式のログを出力する
※python-json-loggerのインストールが必要

In [ ]:
!pip install python-json-logger

In [ ]:
from logging import (
    getLogger,
    StreamHandler,
    FileHandler,
    Formatter,
    DEBUG,
    INFO,
    WARNING,
    ERROR,
    CRITICAL,
)
from pythonjsonlogger import jsonlogger

logger = getLogger(__name__)
# ロガーのログレベルを設定(ハンドラーよりも低い数値(DEBUGに近い)のログレベルにする必要がある)
logger.setLevel(DEBUG)

# ※※JSONフォーマットを設定※※
formatter = jsonlogger.JsonFormatter(
    "%(asctime)s %(levelname)s %(name)s %(funcName)s %(message)s %(filename)s %(lineno)d",
    json_ensure_ascii=False,
)

# 標準出力用ハンドラーのログレベルを設定
s_handler = StreamHandler()
s_handler.setLevel(DEBUG)
s_handler.setFormatter(formatter)
logger.addHandler(s_handler)

# ファイル出力用ハンドラーのログレベルを設定
f_handler = FileHandler(filename="./log/app.log", encoding="utf-8")
f_handler.setLevel(DEBUG)
f_handler.setFormatter(formatter)
logger.addHandler(f_handler)

# ログを出力
logger.critical({"user_name": "tanaka", "message": "クリティカルのメッセージ"})
logger.error({"user_name": "tanaka", "message": "エラーのメッセージ"})
logger.warning({"user_name": "tanaka", "message": "ワーニングのメッセージ"})
logger.info({"user_name": "tanaka", "message": "インフォのメッセージ"})
logger.debug({"user_name": "tanaka", "message": "デバッグのメッセージ"})

## ログの設定をyamlから取得する
※pyyamlのインストールが必要

In [ ]:
!pip install pyyaml

### 一般的な従来の1件1行のログを出力する

In [5]:
from logging import getLogger, config
import yaml

# ログの設定を読み込み
config.dictConfig(
    yaml.load(
        open("./config/logging_nomal_format.yaml").read(),
        Loader=yaml.SafeLoader,
    )
)

# ロガーを取得
logger = getLogger(__name__)

# ログを出力
logger.critical("クリティカルのメッセージ")
logger.error("エラーのメッセージ")
logger.warning("ワーニングのメッセージ")
logger.info("インフォのメッセージ")
logger.debug("デバッグのメッセージ")

### JSON形式のログを出力する

In [6]:
from logging import getLogger, config
import yaml

# ログの設定を読み込み
config.dictConfig(
    yaml.load(
        open("./config/logging_json_format.yaml").read(),
        Loader=yaml.SafeLoader,
    )
)

# ロガーを取得
logger = getLogger(__name__)

# ログを出力
logger.critical({"user_name": "tanaka", "message": "クリティカルのメッセージ"})
logger.error({"user_name": "tanaka", "message": "エラーのメッセージ"})
logger.warning({"user_name": "tanaka", "message": "ワーニングのメッセージ"})
logger.info({"user_name": "tanaka", "message": "インフォのメッセージ"})
logger.debug({"user_name": "tanaka", "message": "デバッグのメッセージ"})

## 関数の開始終了時刻を記録するデコレータの例

In [ ]:
def record_function_start_and_end_time(func):
    def _record_function_start_and_end_time(*args, **kwargs):
        function_name = func.__name__
        logger.debug({"message": f"{function_name} is started."})
        result = func(*args, **kwargs)
        logger.debug({"message": f"{function_name} is completed."})
        return result

    return _record_function_start_and_end_time


@record_function_start_and_end_time
def test_get_double(number: int) -> int:
    return number * 2


print(test_get_double(10))

# トレースを取得する

## opentelemetryでトレースを取得する

In [ ]:
!pip install opentelemetry-api
!pip install opentelemetry-exporter-otlp

### トレースを標準出力に出力する

In [ ]:
from opentelemetry import trace
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import (
    BatchSpanProcessor,
    ConsoleSpanExporter
)
from opentelemetry.sdk.trace.sampling import TraceIdRatioBased

trace_provider = TracerProvider(
    sampler=TraceIdRatioBased(1/1) # トレースを取得する確率を指定
)

trace.set_tracer_provider(trace_provider)
trace.get_tracer_provider().add_span_processor(BatchSpanProcessor(
    ConsoleSpanExporter() # 標準出力用のエクスプローラ
))
tracer = trace.get_tracer(__name__)

In [ ]:
def func1():
    with tracer.start_as_current_span("test1"):
        print("This is func1.")
        func2()

def func2():
    with tracer.start_as_current_span("test2"):
        print("This is func2.")

func1()     

### トレースをopentelemetry-collectorサーバにアップロードする
opentelemetry-collectorサーバはDockerで構築  
`docker run --name my_otel_collector -p 4317:4317 -p 4318:4318 -v .\/config\/opentelemetry-collector:/etc/otelcol otel/opentelemetry-collector`  
(本当は-vオプションでconfig.yamlファイルを指定してマウントしたいが、ファイルを指定するとエラーになる。Dockerの一時的なバグ？)  
  
traceの確認方法は  
`docker logs my_otel_collector`

参考  
https://opentelemetry-python.readthedocs.io/en/latest/exporter/otlp/otlp.html#usage

In [1]:
from opentelemetry import trace
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.trace.export import BatchSpanProcessor
from opentelemetry.sdk.resources import Resource
from opentelemetry.sdk.trace.sampling import TraceIdRatioBased

# 一部のエンドポイントにアップロードするにはサービス名の設定が必要らしい
resource = Resource(attributes={
    "service.name": "my_service"
})

trace.set_tracer_provider(TracerProvider(
    resource=resource,
    sampler=TraceIdRatioBased(1/1) # トレースを取得する確率を指定
))
tracer = trace.get_tracer(__name__)

# サーバ転送用のエクスプローラ
otlp_exporter = OTLPSpanExporter(
    endpoint="http://localhost:4318/v1/traces"
)
span_processor = BatchSpanProcessor(otlp_exporter)
trace.get_tracer_provider().add_span_processor(span_processor)

In [ ]:
def func1():
    with tracer.start_as_current_span("test1"):
        print("This is func1.")
        func2()

def func2():
    with tracer.start_as_current_span("test2"):
        print("This is func2.")

func1()     

### トレース記録用デコレータの例

In [2]:
def otel_trace_decorator(func):
    def _otel_trace_decorator(*args, **kwargs):
        function_name = func.__name__
        with tracer.start_as_current_span(function_name):
            result = func(*args, **kwargs)
        
        return result

    return _otel_trace_decorator

@otel_trace_decorator
def func3():
    print("This is func3.")
    func4()

@otel_trace_decorator
def func4():
    print("This is func4.")

func3()     

This is func3.
This is func4.
